#LCEL Querying a SQL DB

In [ ]:
!pip install cohere tiktoken langchain langchain-experimental openai langchainhub pymssql sqlalchemy

#MSSQL
##DEV
###database_user = 'wonikadmin'
###database_password = 'wonikqnc@6139'
###database_server = 'wiq-qms-sql.database.windows.net'
###database_db = 'WIQ-QMS-DEV-DB'


##PROD
###database_user = 'wonikadmin'
###database_password = 'wonikqnc@6139'
###database_server = 'wiq-qms-sql.database.windows.net'
###database_db = 'WIQ-QMS-PROD-DB'

###1.IMPORT LIBRARY

In [ ]:
from langchain.chat_models import ChatOpenAI

from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA
from langchain.agents import Tool
from langchain.llms import OpenAI
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import AgentType, initialize_agent

from langchain.schema import SystemMessage
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

from langchain.prompts import PromptTemplate,ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
import os

###2.LLM Model

In [ ]:
# ChatGPT 모델 지정
llm = ChatOpenAI(model_name="gpt-4-0613", temperature=0)
#llm = ChatOpenAI(model_name='gpt-3.5-turbo-1106',temperature=0)

###3.KEY SETTINGS

In [ ]:
os.environ["OPENAI_API_KEY"]      = "OPENAI_API_KEY"
os.environ["GOOGLE_API_KEY"]      = "GOOGLE_API_KEY"
os.environ["GOOGLE_CSE_ID"]       = "GOOGLE_CSE_ID"

###4.DATA SOURCE

In [ ]:
#Datasource
database_user = 'USER_NAME'
database_password = 'DB_USER_PW'
database_server = 'DB Server'
database_db = 'Database Name'

####4-1.Connection Str 생성

In [ ]:
#Connection String
import urllib.parse
encoded_password = urllib.parse.quote(database_password)

#1433 : port
connection_string = f"mssql+pymssql://{database_user}:{encoded_password}@{database_server}:1433/{database_db}"

####4-2.Target Table

In [ ]:
#Include tables
include_tables=[ "TB_MST_ITEM","TB_MST_CODE" ]

##5.AGENT TYPE


###5-1.LCEL Chain Type

In [ ]:
#dataSource
db = SQLDatabase.from_uri(connection_string, include_tables=include_tables)

In [ ]:
toolkit = SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0))

In [ ]:
#Utility function
def get_schema(_):
    return db.get_usable_table_names()

def run_query(query):
    return db.run(query)

In [ ]:
from langchain.prompts import ChatPromptTemplate

template = """Based on the table schema below, write a MSSQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
db.get_table_names()

In [ ]:
db.get_table_info()

In [ ]:
sql_response = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

In [ ]:
sql_response.invoke({"question": "How many items are there?"})

In [ ]:
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_template(template)

In [ ]:
full_chain = (
    RunnablePassthrough.assign(query=sql_response)
    | RunnablePassthrough.assign(
        schema=get_schema,
        response=lambda x: db.run(x["query"]),
    )
    | prompt_response
    | llm
)

In [ ]:
full_chain.invoke({"question": "How many items are there?"})

In [ ]:
from langchain.agents import AgentType, initialize_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.utilities import SQLDatabase
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL

###5-2.Legacy Agent Type(initialize_agent)

In [ ]:
#dataSource
db = SQLDatabase.from_uri(connection_string, include_tables=include_tables)

In [ ]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [ ]:
python_repl = PythonREPL()
repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run,
)

custom_tool_list = [repl_tool]

In [ ]:
agent_template = """
  You are an expert MSSQL data analyst.You must query using mssql syntax.
  Be sure to answer in Korean!

  {memory}
  Human: {human_input}
Chatbot:"""

agent_prompt = PromptTemplate(input_variables=["memory", "human_input"],template=agent_template)

agent_memory = ConversationBufferMemory(memory_key="memory",prompt=agent_prompt, return_messages=True)

agent_kwargs = {
            "extra_prompt_messages": [MessagesPlaceholder(variable_name="agent_memory")],
        }

In [ ]:
agent = initialize_agent(
    toolkit.get_tools(),
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,   #ZERO_SHOT_REACT_DESCRIPTION     OPENAI_FUNCTIONS
    memory=agent_memory,
    verbose=True,
    agent_kwargs=agent_kwargs,
    extra_tools=custom_tool_list,
)

In [ ]:
response = agent.run("What is the most popular item? and Show it in chart form")

In [ ]:
response

###5-3.Legacy Agent Type(create_sql_agent)

In [ ]:
db = SQLDatabase.from_uri(connection_string, include_tables=include_tables)

db = SQLDatabase.from_uri(
    "sqlite:///Chinook.db",
    include_tables=[
        "Track"
    ],  # we include only one table to save tokens in the prompt :)
    sample_rows_in_table_info=2,
)

In [ ]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [ ]:
from langchain.prompts import PromptTemplate

custom_suffix = """
You must query using MSSQL.
Be sure to answer in Korean
"""

In [ ]:
agent_template = """
  You are an expert MSSQL data analyst.You must query using mssql syntax.
  Be sure to answer in Korean!

  {memory}
  Human: {human_input}
Chatbot:"""

agent_prompt = PromptTemplate(input_variables=["memory", "human_input"],template=agent_template)

agent_memory = ConversationBufferMemory(memory_key="memory",prompt=agent_prompt, return_messages=True)

agent_kwargs = {
            "extra_prompt_messages": [MessagesPlaceholder(variable_name="agent_memory")],
        }

In [ ]:
# conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

In [ ]:
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,   #ZERO_SHOT_REACT_DESCRIPTION     OPENAI_FUNCTIONS
    handle_parsing_errors=False,
    memory=conversational_memory,
    agent_kwargs=agent_kwargs,
    #suffix=custom_suffix,
)

In [ ]:
response = agent_executor.run("What is the most popular item you are using?")

In [ ]:
response